# 爬取充电站网站

In [45]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import requests
import xlsxwriter
 
# 1.得到一个城市所有停车桩链接 
info = []

options=webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
driver=webdriver.Chrome(options)

driver.get("https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcMap.jsp?chargingTypeId=&companyId=&chargingBrandId=&brandStatuId=&cityName=%E4%B8%8A%E6%B5%B7")  #用浏览器打开链接，北京可以改成其他省份
driver.switch_to.frame('left')       #网页是嵌套的，在一网页下面，有一个iframe，我们找到这个iframe
soup = driver.find_elements(By.TAG_NAME, "a")  #链接都在a标签中，我们找到a标签
for item in soup:
   info.append(item.get_attribute('href')) #提取数a标签里面的href链接
 

In [47]:
len(info)

3103

In [50]:
info[0]

'https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=4494'

In [76]:
#2.爬虫函数
def get_data(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}
        response = requests.get(url, headers=headers, verify=False)
        if response.status_code == 200:
            return response.text
        else:
            print(response.status_code)
            return None
    except:
        print('访问 http 发生错误... ')
        return None

In [193]:
# 3.数据筛选
j=1
workbook = xlsxwriter.Workbook("test.xlsx")
df = workbook.add_worksheet()
cols = ["名称","地址","快充数量","慢充数量","支付方式","充电费","服务费","停车费","开放时间","链接"]
for i in range(10):
   df.write(0,i,cols[i])
   
for u in info:
    url = u
    print(url)
    html = get_data(url)
    if html !=None:
        soup = BeautifulSoup(html)   
        Name = soup.find_all("div",{"class":"news-top"})[0]
        name = Name.find_all("p")[0].text
        Address = soup.find_all("div",{"class":"news-a"})[0]
        address = Address.find_all("p")[0].text

        # 快慢充数量提取
        text = str(soup.find_all("div",{"class":"news-c"})[0].find_all("p"))
        # 匹配快充数量
        fast_charge_pattern = re.compile(r'快充数量：(\d+)个')
        fast_charge_match = fast_charge_pattern.search(text)
        # 匹配慢充数量
        slow_charge_pattern = re.compile(r'慢充数量：(\d+)个')
        slow_charge_match = slow_charge_pattern.search(text)
        # 提取匹配结果，如果没有匹配到则为0
        fast_charge_count = int(fast_charge_match.group(1)) if fast_charge_match else 0
        slow_charge_count = int(slow_charge_match.group(1)) if slow_charge_match else 0

        Pay_type = soup.find_all("div",{"class":"news-con"})[0].find_all("ul",{"class":"news-d details"})[0]

        # 支付方式
        pay_type = Pay_type.find_all("li")[0].find_all("div")[0].text
        pay_type_m = re.compile(r'支付方式：(.+)')
        match = pay_type_m.search(pay_type)
        if match:
            payment_method = match.group(1).strip()
        else:
            payment_method = '不详'

        # 电费
        charge_fee = Pay_type.find_all("li")[1].find_all("div")[0].text
        charge_fee_m = re.compile(r'电费:([^元]+)(?:元|$)')
        match = charge_fee_m.search(charge_fee)
        if match:
            charge_fee_res = match.group(1)
        else:
            charge_fee_res = '不详'

        # 服务费
        service_fee = Pay_type.find_all("li")[2].find_all("div")[0].text
        service_fee_m = re.compile(r'服务费:([^元]+)(?:元|$)')
        match = service_fee_m.search(service_fee)
        if match:
            service_fee_res = match.group(1)
        else:
            service_fee_res = '不详'

        # 停车费
        park_fee = Pay_type.find_all("li")[3].find_all("div")[0].text
        park_fee_m = re.compile(r'停车收费，(.+)$')
        match = park_fee_m.search(park_fee)
        if match:
            park_fee_res = match.group(1)
        else:
            park_fee_res = '不详'
        
        # 开放时间
        open_time_res = Pay_type.find_all("li")[4].find_all("div")[0].text.strip()

        #4.把数据存储
        if name != None:
            df.write(j,0,name)
            df.write(j,1,address)
            df.write(j,2,fast_charge_count)
            df.write(j,3,slow_charge_count)
            df.write(j,4,payment_method)
            df.write(j,5,charge_fee_res)
            df.write(j,6,service_fee_res)
            df.write(j,7,park_fee_res)
            df.write(j,8,open_time_res)
            df.write(j,9,url)
            j = j+1
        else:
            j=j+1

workbook.close()
 

https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=4494
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=6056
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=7058
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=9550
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=9645
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=17551
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=18127
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=21568
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=21947
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=23167
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=42526
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charingId=42527
https://admin.bjev520.com/jsp/beiqi/pcmap/do/pcmap_Detail.jsp?charing

# 地址匹配经纬度

In [220]:
import pandas as pd
import numpy as np
import requests
import json

In [208]:
df = pd.read_excel('test.xlsx')
adss = df['地址'].values.tolist()

In [238]:
adss[2532]

'上海乐购真北店'

In [243]:
for ads in adss[2532:]:
    url='https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=' + ads + '&city=上海'
    print(url)
    res = requests.get(url).content;
    typejson = json.loads(res);

    try:
        district = typejson['geocodes'][0]['district']
        location = typejson['geocodes'][0]['location']
    except (json.JSONDecodeError, KeyError, IndexError):
        district = np.nan
        location = np.nan
    
    df.loc[df['地址'] == ads, 'location'] = location

    if district!=[]:
        df.loc[df['地址'] == ads, 'district'] = district
    else:
        df.loc[df['地址'] == ads, 'district'] = np.nan

https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海乐购真北店&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市宝山区宝山城市工业园区园山连路999号&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市嘉定区马陆镇兴平路669号&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市松江区车墩镇留业路29号&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市嘉定区11号线嘉定西站西区地下车库A区&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市市辖区浦东新区秀沿路1670弄9号&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市松江区佘山镇上海世茂精灵之城主题乐园(东门)上海佘山世茂洲际酒店&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市市辖区浦东新区康威路299号&city=上海
https://restapi.amap.com/v3/geocode/geo?key=55d2898ef852cc85b7fbaba29970db37&address=上海市嘉定区安辰路

In [244]:
df.to_csv('sh_charging_station_info.csv')